### 构建卷积神经网络
- 卷积网络中的输入和层与传统神经网络有些区别，需重新设计，训练模块基本一致

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets,transforms 
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

### 首先读取数据
- 分别构建训练集和测试集（验证集）
- DataLoader来迭代取数据

In [3]:
# 定义超参数 
input_size = 28  #图像的总尺寸28*28
num_classes = 10  #标签的种类数
num_epochs = 3  #训练的总循环周期
batch_size = 64  #一个撮（批次）的大小，64张图片

# 训练集
train_dataset = datasets.MNIST(root='./data',  
                            train=True,   
                            transform=transforms.ToTensor(),  
                            download=True) 

# 测试集
test_dataset = datasets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

# 构建batch数据
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

### 卷积网络模块构建
- 一般卷积层，relu层，池化层可以写成一个套餐
- 注意卷积最后结果还是一个特征图，需要把图转换成向量才能做分类或者回归任务

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         # 输入大小 (1, 28, 28)
            nn.Conv2d(
                in_channels=1,              # 灰度图
                out_channels=16,            # 要得到几多少个特征图
                kernel_size=5,              # 卷积核大小
                stride=1,                   # 步长
                padding=2,                  # 如果希望卷积后大小跟原来一样，需要设置padding=(kernel_size-1)/2 if stride=1
            ),                              # 输出的特征图为 (16, 28, 28)
            nn.ReLU(),                      # relu层
            nn.MaxPool2d(kernel_size=2),    # 进行池化操作（2x2 区域）, 输出结果为： (16, 14, 14)
        )
        self.conv2 = nn.Sequential(         # 下一个套餐的输入 (16, 14, 14)
            nn.Conv2d(16, 32, 5, 1, 2),     # 输出 (32, 14, 14)
            nn.ReLU(),                      # relu层
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(2),                # 输出 (32, 7, 7)
        )
        
        self.conv3 = nn.Sequential(         # 下一个套餐的输入 (16, 14, 14)
            nn.Conv2d(32, 64, 5, 1, 2),     # 输出 (32, 14, 14)
            nn.ReLU(),             # 输出 (32, 7, 7)
        )
        
        self.out = nn.Linear(64 * 7 * 7, 10)   # 全连接层得到的结果

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        #  将张量 x 从多维形状变换为一个二维形状，其中第一个维度是 batch_size，而第二个维度是 -1。 -1 的意思是根据张量的总元素数自动计算该位置的维度，以保持原有元素数不变。
        # 在这里，x.size(0) 表示 batch_size，因此该操作将每个样本的特征张量展平为一维向量，保持批次大小不变。具体地，由于之前的卷积和池化操作，x 的形状是 (batch_size, 64, 7, 7)，通过 view 
        # 作后，形状变为 (batch_size, 64 * 7 * 7)，即每个样本的特征被展平为一个长度为 64 * 7 * 7 的一维向量。
        # 这种形状变换通常在卷积神经网络的卷积层之后、全连接层之前使用，以适应全连接层的输入要求，因为全连接层需要一维的输入。
        x = x.view(x.size(0), -1)           # flatten操作，结果为：(batch_size, 32 * 7 * 7)
        output = self.out(x)
        return output

### 准确率作为评估标准

In [5]:
def accuracy(predictions, labels):
    # torch.max 函数用于在指定维度上返回最大值。
    # predictions.data 提取模型的预测值的数据部分。
    # 1 表示沿着第一个维度（通常是列）寻找最大值，即对每个样本找到具有最大值的类别。
    # 返回的结果包含两个张量：最大值的张量和最大值所在的索引张量。
    pred = torch.max(predictions.data, 1)[1] 
    # 对预测的类别标签与真实标签进行逐元素比较，生成一个布尔值张量，相等的位置为 True，不相等的位置为 False。
    # sum() 对布尔值张量求和，得到相等的数量。这表示模型正确预测的样本数量
    rights = pred.eq(labels.data.view_as(pred)).sum() 
    # rights 是正确预测的样本数量。
    # len(labels) 是总样本数量。
    return rights, len(labels) 

### 训练网络模型

In [6]:
# 实例化
net = CNN() 
#损失函数
criterion = nn.CrossEntropyLoss() 
#优化器
optimizer = optim.Adam(net.parameters(), lr=0.001) #定义优化器，普通的随机梯度下降算法

#开始训练循环
for epoch in range(num_epochs):
    #当前epoch的结果保存下来
    train_rights = [] 
    
    for batch_idx, (data, target) in enumerate(train_loader):  #针对容器中的每一个批进行循环
        net.train()                             
        output = net(data) 
        loss = criterion(output, target) 
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step() 
        right = accuracy(output, target) 
        train_rights.append(right) 

    
        if batch_idx % 100 == 0: 
            
            net.eval() 
            val_rights = [] 
            
            for (data, target) in test_loader:
                output = net(data) 
                right = accuracy(output, target) 
                val_rights.append(right)
                
            #准确率计算
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))

            print('当前epoch: {} [{}/{} ({:.0f}%)]\t损失: {:.6f}\t训练集准确率: {:.2f}%\t测试集正确率: {:.2f}%'.format(
                epoch, batch_idx * batch_size, len(train_loader.dataset),
                100. * batch_idx / len(train_loader), 
                loss.data, 
                100. * train_r[0].numpy() / train_r[1], 
                100. * val_r[0].numpy() / val_r[1]))

当前epoch: 0 [0/60000 (0%)]	损失: 2.303364	训练集准确率: 9.38%	测试集正确率: 10.09%
当前epoch: 0 [6400/60000 (11%)]	损失: 0.250181	训练集准确率: 76.95%	测试集正确率: 92.50%
当前epoch: 0 [12800/60000 (21%)]	损失: 0.078053	训练集准确率: 85.70%	测试集正确率: 95.46%
当前epoch: 0 [19200/60000 (32%)]	损失: 0.135206	训练集准确率: 89.17%	测试集正确率: 97.29%
当前epoch: 0 [25600/60000 (43%)]	损失: 0.142493	训练集准确率: 91.07%	测试集正确率: 97.79%
当前epoch: 0 [32000/60000 (53%)]	损失: 0.059492	训练集准确率: 92.36%	测试集正确率: 97.91%
当前epoch: 0 [38400/60000 (64%)]	损失: 0.069894	训练集准确率: 93.21%	测试集正确率: 98.23%
当前epoch: 0 [44800/60000 (75%)]	损失: 0.017419	训练集准确率: 93.90%	测试集正确率: 98.25%
当前epoch: 0 [51200/60000 (85%)]	损失: 0.060239	训练集准确率: 94.41%	测试集正确率: 98.62%
当前epoch: 0 [57600/60000 (96%)]	损失: 0.095850	训练集准确率: 94.86%	测试集正确率: 98.33%
当前epoch: 1 [0/60000 (0%)]	损失: 0.046640	训练集准确率: 98.44%	测试集正确率: 98.18%
当前epoch: 1 [6400/60000 (11%)]	损失: 0.009855	训练集准确率: 98.14%	测试集正确率: 98.65%
当前epoch: 1 [12800/60000 (21%)]	损失: 0.017181	训练集准确率: 98.40%	测试集正确率: 98.46%
当前epoch: 1 [19200/60000 (32%)]	损失: 0.092074	训练集准确率:

### 练习
- 再加入一层卷积，效果怎么样？
- 当前任务中为什么全连接层是32*7*7 其中每一个数字代表什么含义